# Приложение: сравнение методов оценки А/Б-тестов

*Проведено сравнение методов выбора лучше группы в А/Б-тестах по количеству правильно угаданных вариантов*.

* [Введение](#Введение)
* [Способы выбора лучшей группы в А/Б-тесте](#Способы-выбора-лучшей-группы-в-А/Б-тесте)    
&nbsp;&nbsp; - [Сравнение средних в сэмплах](#Сравнение-средних-в-сэмплах)  
&nbsp;&nbsp; - [Оценка разности средних и выбор лучшей группы с помощью неравенства Чебышева](#Оценка-разности-средних-и-выбор-лучшей-группы-с-помощью-неравенства-Чебышева)  
&nbsp;&nbsp; - [Оценки разности средних с учетом центральной предельной теоремы](#Оценки-разности-средних-с-учетом-центральной-предельной-теоремы)  
&nbsp;&nbsp; - [Метод максимального правдоподобия](#Метод-максимального-правдоподобия)  
&nbsp;&nbsp; - [Байесовское моделирование](#Байесовское-моделирование)  
&nbsp;&nbsp; - Проверка статистических гипотез  
&nbsp;&nbsp; - [Бутстрап](#Бутстрап)  
* [Количество правильно угаданных вариантов](#Количество-правильно-угаданных-вариантов)
* Выводы

# Введение

Есть разные методы выбора лучшей группы в А/Б-тесте.  
Интересно сравнить их на модельных примерах - какие методы чаще и за меньшее количество данных угадывают лучшие варианты.   

Также полезно сравнить, как в рамках различных методов даются ответы на вопросы:
- Какой вариант лучше и насколько?
- Каковы оценки целевой метрики в каждом варианте?
- Насколько уверены в оценке?
- Сколько должен продолжаться эксперимент?

# Способы выбора лучшей группы в А/Б-тесте

Пусть есть 2 распределения случайных величин одинаковой формы, но с разными параметрами.  
Загадываются параметры, генерируются выборки.  
По выборкам нужно сделать оценку параметров и выбрать "лучшую" группу.  

Для характеристики распределений часто проще использовать ожидаемое значение.  
Т.е. по выборкам нужно определить распределение с большим средним.

Конверсии часто моделируют независимыми испытаниями с двумя исходами (испытания Бернулли)
[[Bern](https://en.wikipedia.org/wiki/Bernoulli_process)].   
Выручку на платящего пользователя иногда моделируют распределением Парето.   
Реальное поведение пользователей и метрик может требовать более сложных распределений.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

In [ ]:
n = 100
p_a_exact = 0.3
sa = stats.binom.rvs(n, p_a_exact)
p_b_exact = 0.33
sb = stats.binom.rvs(n, p_b_exact)

fig = go.Figure()
fig.add_trace(go.Bar(x=[1, 0], y=[sa, n-sa], name='Sample A', marker_color='red'))
fig.add_trace(go.Bar(x=[1, 0], y=[sb, n-sb], name='Sample B', marker_color='blue'))
fig.add_trace(go.Scatter(x=[p_a_exact, p_a_exact], y=[0, max(sa, n-sa)], 
                         line_dash='dash', mode='lines', name='Exact Success Prob A', line_color='red'))
fig.add_trace(go.Scatter(x=[p_b_exact, p_b_exact], y=[0, max(sb, n-sb)], 
                         line_dash='dash', mode='lines', name='Exact Success Prob B', line_color='blue'))
fig.update_layout(
    title='Bernoulli Sequence Samples and Exact Success Probabilities for Two Groups',
    xaxis_range=[-0.1, 1.1],
    barmode='group', bargap=0.9, bargroupgap=0.0,
    height=500, width=850)
fig.show()

In [ ]:
n = 1000

c_a_exact = 2.5
exact_dist_a = stats.lomax(c=c_a_exact)
samp_a = exact_dist_a.rvs(size=n)

c_b_exact = 2.8
exact_dist_b = stats.lomax(c=c_b_exact)
samp_b = exact_dist_b.rvs(size=n)

# mean = 1 / (c - 1), c > 1

# x = np.linspace(0, 10, 2000)
# ymax = np.max([exact_dist_a.pdf(x), exact_dist_b.pdf(x)])
ymax = 1

fig = go.Figure()
col = 'red'
# fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pdf(x), 
#                          mode='lines', line_color=col,
#                          name=f"Exact A: a={exp_info['a']['A']}, b={exp_info['b']['A']}"))
fig.add_trace(go.Histogram(x=samp_a, histnorm='probability density',
                           name='Sample A',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_a.mean(), exact_dist_a.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean A'))
col = 'blue'
# fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pdf(x), 
#                          mode='lines', line_color=col,
#                          name=f"Exact B: a={exp_info['a']['B']}, b={exp_info['b']['B']}"))
fig.add_trace(go.Histogram(x=samp_b, histnorm='probability density',
                           name='Sample B',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_b.mean(), exact_dist_b.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean B'))
fig.update_layout(title='Pareto Exact Distributions and Samples',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

## Сравнение средних в сэмплах

По закону больших чисел среднее в выборке сходится к среднему в распределении при росте количества элементов в выборке [[LLN](https://en.wikipedia.org/wiki/Law_of_large_numbers)]

$$
\lim_{n \to \infty} \sum_{i=1}^{n} \frac{X_i}{n} = \bar{X} .
$$

Сходимость понимается как сходимость по распределению (слабый закон больших чисел).  
*Иначе можно сказать, что среднее в сэмпле - состоятельная несмещенная оценка среднего в распределении* https://en.wikipedia.org/wiki/Estimator .

Сравнение средних в выборке с точным средним по мере роста количества точек выборки представлено на графике ниже. *Видно, что при размере выборки в несколько сот точек приближение отличия в пределах 5%, а при выбранных параметрах при $n > 1000$ относительное отличие не превышает 5%.* 

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.6

n = np.arange(10, 5000, 100)
k_a = stats.binom.rvs(n, p_a_exact)
k_b = stats.binom.rvs(n, p_b_exact)
p_a_samp = k_a/n
p_b_samp = k_b/n

fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=p_a_samp, name='A: p sample'))
fig.add_trace(go.Scatter(x=[0, np.max(n)], y=[p_a_exact, p_a_exact],
                         mode='lines',
                         line_dash='dash', line_color='black', name='A: p exact'))
fig.add_trace(go.Scatter(x=n, y=p_b_samp, name='B: p sample'))
fig.add_trace(go.Scatter(x=[0, np.max(n)], y=[p_b_exact, p_b_exact],
                         mode='lines',
                         line_dash='dash', line_color='black', name='B: p exact'))
fig.update_layout(
    title='Exact and Sample Means for Binomial Distribution',
    xaxis_title='Sample Size',
    yaxis_title='p',
    yaxis_range=[0, 1],
    xaxis_range=[0, np.max(n)],
    height=550
)
fig.show()

Поскольку выборочные средние приближают среднее в распределении, то для выбора распределения с большим средним можно выбрать группу с большим средним в выборке.

In [ ]:
def compare_binomial_sample_means(sa, na, sb, nb):
    pa = sa / na
    pb = sb / nb
    if pa > pb:
        res = 'A'
    elif pa < pb:
        res = 'B'
    else:
        res = None
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.75

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by sample means comparison: {compare_binomial_sample_means(sa, n, sb, n)}')

Предложенный метод позволяет сделать оценки интересующих метрик и выбрать одну из групп в А/Б-тесте, но не дает оценки вероятности, с какой одна группа лучше другой. Также не обсуждался вопрос о длительности эксперимента.

*todo: посмотреть скорость сходимости. Если она считается иначе, чем в центральной предельной теореме, можно попробовать получить оценку длительности из нее*

| Метод                 | Оценки целевой метрики                              | Критерий выбора группы   | Оценка длительности |
|-----------------------|-----------------------------------------------------|--------------------------|---------------------|
| **Сравнение средних** | Точечные оценки средних на основе средних в выборке | Больше среднее в выборке | (?)                 |

## Оценка разности средних и выбор лучшей группы с помощью неравенства Чебышева

* Посчитать средние и дисперсии в каждой группе
* Посчитать стандартные ошибки средних
* С помощью неравенства Чебышева оценить интервал возможных значений
* Построить разность средних и дисперсию разности средних
* Посчитать вероятность разности больше 0. 

$$
A = [1, 0, 1, 0, 0, ... , 1]
\\
s_A, n_A
\\
E_A \equiv p_A = s_A / n_A
\\
D_A = p_A (1 - p_A)
\\
\tilde{p}_A \equiv \frac{1}{n} \left( \tilde{A} + \tilde{A} + \dots + \tilde{A} \right)
\\
E[\tilde{p}_A] = E_A = p_A
\\
D[\tilde{p}_A] = \frac{D_A}{n_A}
\\
\Delta = \tilde{p}_A - \tilde{p}_B
\\
E_{\Delta} = E[\tilde{p}_A - \tilde{p}_B] = E[\tilde{p}_A] - E[\tilde{p}_B] = p_A - p_B
\\
D_{\Delta} = D[\tilde{p}_A - \tilde{p}_B] = D[\tilde{p}_A] + D[\tilde{p}_B]
\\
\sigma_{\Delta} = \sqrt{D[p_A] + D[p_B]}
\\
P(|0 - E_{\Delta}| \ge k \sigma_{\Delta}) \le \frac{1}{k^2}
\\
P \left( \frac{|E_{\Delta}|}{\sigma_{\Delta}} \ge k \right) \le \frac{1}{k^2}
$$

In [ ]:
p_a_exact = 0.3
n = 1000
sa = stats.binom.rvs(n, p_a_exact)

p_samp = sa / n
stderr = np.sqrt(p_samp * (1-p_samp) / n)

fig = go.Figure()
fig.add_trace(go.Bar(x=[1, 0], y=[sa, n-sa], width=0.1, name='Sample'))
fig.add_trace(go.Scatter(x=[p_a_exact, p_a_exact], y=[0, max(sa, n-sa)], 
                         line_dash='solid', mode='lines', name='Exact Mean'))
xtmp = np.arange(0, 1, 0.001)
ytmp = stats.norm.pdf(x=xtmp, loc=p_samp, scale=stderr)
ytmp = ytmp / max(ytmp) * max(sa, n-sa)
fig.add_trace(go.Scatter(x=xtmp, y=ytmp,
                        line_dash='dash', name='Mean Estimate from Sample'))
fig.update_layout(
    xaxis_range=[0,1],
    height=500, width=850)
fig.show()

Оценку среднего значения неизвестного распределения по выборке можно рассматривать как построение оценки случайной величины. 
Предыдущий метод давал точечную оценку среднего, но не рассматривал его неопределенность.

Если исходное распределение с конечной диспресией, то 
дисперсия выборочных средних задается выражением 
(стандартная ошибка среднего [[StErrMn](https://en.wikipedia.org/wiki/Standard_error#Derivation)])

$$
D[{\bar{x}}] = D \left[ \frac{1}{n} \sum_{i=1}^n x_i \right] = \frac{1}{n^2} n D[x] = \frac{\sigma^2}{n} ,
$$

где $\sigma$ - стандартное отклонение исходного распределения, $n$ - количество точек в выборочном среднем. 

Оценку дисперсии $\sigma$ можно построить по сэмлу.  
Для дисперсии нужны поправки к значению в сэмпле $s$ чтобы получить несмещенную оценку [[StdUnbiased](https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation)]  

$$
\sigma \approx \sqrt{\frac{n}{n-1}} s, \quad s = \sqrt{ \frac{1}{n} \sum_{i=1}^n (x_i - \bar{x}_i)^2 } .
$$

При больших $n$ можно для оценки стандартного отклонения распределения использовать стандартное отклонение в выборке $s$ без корректирующих правок

$$
\sigma \approx s .
$$

Оценить вероятность отклонения случайной величины от среднего можно с помощью неравенства Чебышева 
[[Cheb](https://en.wikipedia.org/wiki/Chebyshev%27s_inequality)]   

$$
P(|X - \mu| \ge k \sigma) \le \frac{1}{k^2} .
$$

Для двух групп можно получить оценки средних, дисперсий и интервалов вероятных значений.  


Для сравнения групп можно посчитать вероятность $P( E[A - B] > 0 )$.  
*todo: ввести обозначение для оценки распределения средних.*  
*что-то типа $\tilde{p}$*  
*тогда нужно оценить $P(\tilde{p}_A - \tilde{p}_B > 0)$*

Среднее разности случайных величин линейно и считается как разность средних [[ExpProp](https://en.wikipedia.org/wiki/Expected_value#Properties)]  

$$
E[X - Y] = E[X] - E[Y] .
$$

Дисперсия разности случайных величин 
[[VarProp](https://en.wikipedia.org/wiki/Standard_deviation#Identities_and_mathematical_properties)] 

$$
D[X - Y] = D[X] + D[Y] - 2cov[X,Y] .
$$

В данных примерах величины X и Y независимы.  
Для независимых величин $cov[X,Y] = 0$ [[Cov](https://en.wikipedia.org/wiki/Covariance)].  
В реальном А/Б-тесте это слагаемое можно оставить.

Таким образом удается оценить ожидаемое значение и стандартное отклонение разности средних двух групп  

$$
E_{\Delta} = E[A] - E[B], 
\quad
\sigma_{\Delta} = \sqrt{D[p_A] + D[p_B]} .
$$

С помощью неравенства Чебышева можно оценить вероятность среднее в одной группе больше другой

$$
P(|0 - E_{\Delta}| \ge k \sigma_{\Delta}) \le \frac{1}{k^2}
\\
P \left( \frac{|E_{\Delta}|}{\sigma_{\Delta}} \ge k \right) \le \frac{1}{k^2}
$$

Одностороннее неравенство Чебышева: https://en.wikipedia.org/wiki/Chebyshev%27s_inequality#Cantelli's_inequality

In [ ]:
def std_err_mean_binom(s, n):
    p = s / n
    return np.sqrt(p * (1 - p) / n)

def compare_binomial_stderr_chebyshev(sa, na, sb, nb, diff_prob=0.9):
    pa = sa / na
    pb = sb / nb
    std_err_mean_a = std_err_mean_binom(sa, na)
    std_err_mean_b = std_err_mean_binom(sb, nb)
    mean_diff = pb - pa
    std_err_mean_diff = np.sqrt(std_err_mean_a**2 + std_err_mean_b**2)
    k = np.abs(mean_diff) / std_err_mean_diff
    prob = 1.0 / k**2
    if pa > pb and prob <= diff_prob:
        res = 'A'
    elif pa < pb and prob <= diff_prob:
        res = 'B'
    else:
        res = None
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.75

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by means comparison: {compare_binomial_stderr_chebyshev(sa, n, sb, n)}')

Метод не делает предположений о форме распределений тестируемых величин. Единственное, что требуется - существование конечных средних и дисперсий. Чаще всего эти предположения можно считать выполненными. Однако при конечных среднем и дисперсии обычно применима центальная предельная теорема, на основании которой можно построить более точную форму распределения и использовать его для оценок вероятностей вместо неравенства Чебышева - см. следующий раздел.

| Метод                                        | Оценки целевой метрики                                                                        | Критерий выбора группы                                 | Оценка длительности |
|----------------------------------------------|-----------------------------------------------------------------------------------------------|--------------------------------------------------------|---------------------|
| **Сравнение средних + неравенство Чебышева** | Точечные оценки средних на основе средних в выборке,  интервалы с помощью неравества Чебышева | Вероятность P(A-B > 0) на основе  неравенства Чебышева | (?)                 |

*Остается вопрос интерпретации вероятности и интервалов вокруг средних значений.*  
*Вводится новая случайная величина - сумма N наблюдений.*  
*Делаются утверждения о ее распределении. К тому же приближенные.*  
*Как это соотносится с исходными вопросами?*

## Оценки разности средних с учетом центральной предельной теоремы

Если у распределения конечная дисперсия, то можно применять центральные предельные теоремы [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)].  
Это дает более строгую оценку на область вероятных значений средних, чем неравенство Чебышева.

Для серий Бернулли см. также теорему Муавра-Лапласа [[DMLT](https://en.wikipedia.org/wiki/De_Moivre%E2%80%93Laplace_theorem)] 

$$
S_n = \xi_1 + ... + \xi_n
\\
a = E[\xi],
\qquad
\sigma^2 = D[\xi]
\\
P\left(\alpha < \frac{S_n - na}{\sqrt{n}\sigma} < \beta \right) \to \frac{1}{2 \pi} \int_{\alpha}^{\beta} e^{-t^2/2} dt 
$$

Центральная предельная теорема позволяет по выборке построить интервал, который будет содержать среднее значение с заданной вероятностью:

$$ 
P\left( \frac{S_n - \sqrt{n}\sigma \beta}{n} < a < \frac{S_n - \sqrt{n}\sigma \alpha}{n} \right) \to \frac{1}{2 \pi} \int_{\alpha}^{\beta} e^{-t^2/2} dt 
$$

Такие конструкции называются доверительными интервалами [[ConfInt](https://en.wikipedia.org/wiki/Confidence_interval)]. Частотная интерпретация этой вероятности - провести эксперимент и получить выборку из n элементов, по выборке построить границы интервала, с заданной вероятность (например, в 95 повторах из 100) реальное среднее будет лежать внутри интервала. 

Для оценки разности можно ввести случайную величину $\xi_{\Delta} = \xi_B - \xi_A$. Для нее использовать выражение выше напрямую.  
Если число точек в группе неодинаковое, это менее удобно.

Можно приближенно считать $\tilde{p}_A$ и $\tilde{p}_B$ распределенными нормально

$$
\tilde{p}_A \sim Norm(p_A, s_A), \quad \tilde{p}_B \sim Norm(p_B, s_B) .
$$

Далее воспользоваться свойством нормальных распределений по которому разность нормальных распределений также будет распределена нормально [[NormSum](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables)]
$$
\tilde{p}_{\Delta} \equiv \tilde{p}_B - \tilde{p}_A ,
\\
\tilde{p}_{\Delta} \sim Norm(p_B - p_A, \sqrt{\sigma_A^2 + \sigma_B^2}) .
$$

In [ ]:
def std_err_mean_binom(s, n):
    p = s / n
    return np.sqrt(p * (1 - p) / n)

def compare_binomial_clt(sa, na, sb, nb, diff_prob=0.9):
    pa = sa / na
    pb = sb / nb
    std_err_mean_a = std_err_mean_binom(sa, na)
    std_err_mean_b = std_err_mean_binom(sb, nb)
    mean_diff = pb - pa
    std_err_mean_diff = np.sqrt(std_err_mean_a**2 + std_err_mean_b**2)
    #
    pb_gt_pa = 1 - stats.norm.cdf(x=0, loc=mean_diff, scale=std_err_mean_diff)
    #
    if pa > pb and diff_prob <= 1 - pb_gt_pa:
        res = 'A'
    elif pa < pb and diff_prob <= pb_gt_pa:
        res = 'B'
    else:
        res = None
    return res

| Метод                                                  | Оценки целевой метрики                                                                                   | Критерий выбора группы                                           | Оценка длительности |
|--------------------------------------------------------|----------------------------------------------------------------------------------------------------------|------------------------------------------------------------------|---------------------|
| **Сравнение средних + центральная предельная теорема** | Точечные оценки средних на основе средних в выборке,  интервалы с помощью центральной предельной теоремы | Вероятность P(A-B > 0) на основе  центральной предельной теоремы | (?)                 |

# Метод максимального правдоподобия

Предыдущие методы не делали предположений о форме распределения сравниваемых случайных величин.  
При учете таких предположений сравнение может быть эффективнее.  

В методе максимального правдоподобия [[MxLk](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation)] делается предположение о распределении случайной величины.  
Эту функцию называют функцией правдоподобия. Обычно у этой функции есть параметы.   
Значения параметров выбирается так, чтобы максимизировать эту функцию при имеющихся данных.  

Для некоторых моделей можно проверси аналитические вычисления.  
Для биномиального распределения максимум правдоподобия совпадет со средним в сэмпле.  

$$
P(s, n) = C^n_s p^s (1-p)^{n-s}
\\
\frac{d P}{dp} = C s p^{s-1} (1-p)^{n-s} - C p^s (n-s) (1-p)^{n-s-1} = 0
\\
C p^{s-1} (1-p)^{n-s-1} \Big[ s (1-p) - p (n-s) \Big] = 0
\\
s - pn = 0
\\
p = \frac{s}{n}
$$

Т.е. когда $n$ бросков монетки и $s$ успехов моделируются биномиальным распределением, то получить пару $(n,s)$ с наибольшей вероятностью можно при значении $p = s/n$.

*Для нормального распределения можно показать, что параметры, максимизирующие функцию правдоподобия, совпадут со средним и дисперсией в сэмле. (?)* 

Выбрана модель и оценены параметры, дальше можно сравнивать эти распределения.  
Если есть возможность посчитать средние по параметрам аналитически, то можно выбрать группу с большим средним.  

Для биномиальных распределений его результаты совпадают с простым сравнением средних.

Метод формально говорит о полной уверенности.

In [ ]:
def compare_binomial_maxlikelihood(sa, na, sb, nb):
    pa = sa / na
    pb = sb / nb
    if pa > pb:
        res = 'A'
    elif pa < pb:
        res = 'B'
    else:
        res = None
    return res

Метод максимального правдоподобия можно рассматривать как частный случай байесовского моделирования.  
В байесовском подходе параметры интерпретируются как случайные величины с предполагаемым распределением.  
В методе максимального правдоподобия - считаются неизвестной постоянной величиной.
Оценка параметров методом максимального правдоподобия совпадет с максимум апостериорного распределения в байесовском моделировании, если априорное распределение равномерное [[MxLk](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation#Relation_to_Bayesian_inference), [MAPE](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation)].  

Метод оценивает точечные оценки параметров, а не распределения.  
Поскольку вместо распределения параметров используется точечная оценка, метод будет давать завышенную уверенность по сравнению со сравнением апостериорных распределений в байесовском подходе.  

| Метод                          | Оценки целевой метрики                                                                                                                | Критерий выбора группы                      | Оценка длительности |
|--------------------------------|---------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------|---------------------|
| **Максимальное правдоподобие** | Предположение формы распределения;  оценка параметров, максимизирующих  функцию правдоподобия; сравнение характеристик распределения. | Сравнение интересующих параметров распределения. | (?)                 |

## Байесовское моделирование

Использование байесовского моделирования для сравнения групп в А/Б-тестах рассмотрено в предыдущих частях.

Основная идея - предположить аналитический вид распределения ("модель"). С помощью соотношения Байеса построить плотность вероятности параметров модели. Сравнивать эти распределения. 

In [ ]:
def posterior_binom_beta(sa, n, alpha=1, beta=1):
    alpha_post = alpha + sa
    beta_post = beta + (n - sa)
    return stats.beta(alpha_post, beta_post)

In [ ]:
pa_exact = 0.7
pb_exact = 0.65

n = 1000
sa = stats.binom.rvs(n, pa_exact)
sb = stats.binom.rvs(n, pb_exact)

post_dist_a = posterior_binom_beta(sa, n)
post_dist_b = posterior_binom_beta(sb, n)

x = np.linspace(0, 1, 1000)
ymax = np.max([post_dist_a.pdf(x), post_dist_b.pdf(x)])
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_dist_a.pdf(x), name='A'))
fig.add_trace(go.Scatter(x=[pa_exact, pa_exact], y=[0, ymax], 
                         mode='lines', line_dash='dash', line_color='black', 
                         name='A: p exact'))
fig.add_trace(go.Scatter(x=x, y=post_dist_b.pdf(x), name='B'))
fig.add_trace(go.Scatter(x=[pb_exact, pb_exact], y=[0, ymax],
                         mode='lines', line_dash='dash', line_color='black', 
                         name='B: p exact'))
fig.update_layout(title='Posterior',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  xaxis_range=[0.5, 1],
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

In [ ]:
def compare_groups_bayes_binomial(sa, na, sb, nb, diff_prob=0.9):
    n_post_sample = 30000
    post_a = posterior_binom_beta(sa, na)
    post_b = posterior_binom_beta(sb, nb)
    post_samp_a = post_a.rvs(n_post_sample)
    post_samp_b = post_b.rvs(n_post_sample)
    pa_gt_pb = np.sum(post_samp_a > post_samp_b) / len(post_samp_a)
    res = None
    if pa_gt_pb >= diff_prob:
        res = 'A'
    elif pa_gt_pb <= (1 - diff_prob):
        res = 'B'
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.65

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by bayesian comparison: {compare_groups_bayes_binomial(sa, n, sb, n)}')

# Количество правильно угаданных вариантов

Можно предложить несколько способов выбора лучшей группы в А/Б-тесте.

In [ ]:
df_methods = pd.DataFrame(columns=[
    'Метод', 
    'Оценки целевой метрики', 
    'Критерий выбора группы', 
    'Оценка длительности'
])
df_methods = pd.concat([df_methods, pd.DataFrame([{
    'Метод': 'Сравнение средних', 
    'Оценки целевой метрики': 'Точечные оценки средних на основе средних в выборке', 
    'Критерий выбора группы': 'Больше среднее в выборке', 
    'Оценка длительности': '(?)'
}])], ignore_index=True)

df_methods = pd.concat([df_methods, pd.DataFrame([{
    'Метод': 'Сравнение средних + неравенство Чебышева', 
    'Оценки целевой метрики': 'Точечные оценки средних на основе средних в выборке, интервалы с помощью неравества Чебышева', 
    'Критерий выбора группы': 'Вероятность P(A-B > 0) на основе неравенства Чебышева', 
    'Оценка длительности': '(?)'
}])], ignore_index=True)

df_methods = pd.concat([df_methods, pd.DataFrame([{
    'Метод': 'Центральная предельная теорема', 
    'Оценки целевой метрики': 'Точечные оценки средних на основе средних в выборке, интервалы по центральной предельной теореме', 
    'Критерий выбора группы': 'Вероятность P(A-B > 0) на основе центральной предельной теоремы', 
    'Оценка длительности': '(?)'
}])], ignore_index=True)

df_methods = pd.concat([df_methods, pd.DataFrame([{
    'Метод': 'Максимальное правдоподобие', 
    'Оценки целевой метрики': 'Предположение формы распределения;  оценка параметров, максимизирующих  функцию правдоподобия; сравнение характеристик распределения.', 
    'Критерий выбора группы': 'Сравнение интересующих параметров распределения', 
    'Оценка длительности': '(?)'
}])], ignore_index=True)

df_methods = pd.concat([df_methods, pd.DataFrame([{
    'Метод': 'Байесовское моделирование', 
    'Оценки целевой метрики': 'Построение апостериорных распределений параметров', 
    'Критерий выбора группы': 'Сравенние апостериорных распределений', 
    'Оценка длительности': '(?)'
}])], ignore_index=True)

with pd.option_context('display.max_colwidth', 0):
    display(df_methods.set_index('Метод'))

Интересно сравнить эти методы по количеству правильно угаданных вариантов.

Первая задача - выбрать биномиальное распределение с большим значением конверсии из двух распределений по выборке фиксированного размера.  

Сравнение будет проводиться следующим образом.  
Выбирается количество наблюдений $N$.  
Для фиксированного количества наблюдений генерируется $k$ пар параметров $p_A, p_B$.  
Для каждой пары генерируется число успехов $s_A, s_B$.  
По известным $s_A, s_B, N$ нужно выбрать группу с большим значением $p$.  
Сравнить выбранную группу с реально лучшей.  

Генерация данных для эксперимента

In [ ]:
def gen_binomial_experiments(n_points, k_experiments, pa_base=0.1):
    pa = np.full(k_experiments, pa_base)
    pb = stats.uniform.rvs(loc=0.9*pa, scale=0.1*pa) #unif[loc, loc+scale]
    sa = stats.binom.rvs(n=n_points, p=pa, size=k_experiments)
    sb = stats.binom.rvs(n=n_points, p=pb, size=k_experiments)
    e = {
        'exp_type': 'binomial',
        'k_experiments': k_experiments,
        'n_points': n_points,
        'pa_exact': pa,
        'pb_exact': pb,
        'sa': sa,
        'sb': sb
    }
    return e

Сравнение экспериментов и подсчет количества правильно угаданных вариантов

In [ ]:
def compare_experiments_binomial_exact(exps):
    s = np.empty_like(exps['pa_exact'], dtype=object)
    s[exps['pa_exact'] > exps['pb_exact']] = 'A'
    s[exps['pb_exact'] > exps['pa_exact']] = 'B'
    res = {
        'comparison_type': 'exact',
        'selected': s
    }
    return res

def compare_experiments(exps, method_name, method_fn):
    s = np.empty(k_experiments, dtype=object)
    n = exps['n_points']
    for i, (sa, sb) in enumerate(zip(exps['sa'], exps['sb'])):
        s[i] = method_fn(sa=sa, na=n, sb=sb, nb=n)
    res = {
        'comparison_type': method_name,
        'selected': s
    }
    return res

def method_guesses(method, exact, n_points):
    e = exact['selected']
    m = method['selected']
    s = {
        'name': method['comparison_type'],
        'n_points': n_points,
        'experiments': len(e),
        'correct': np.sum(e == m),
        'not_sure': len(e) - np.count_nonzero(m),
        'incorrect': np.count_nonzero(m) - np.sum(e == m)
    }
    return s

In [ ]:
n_points = np.array([10, 100, 1000, 5000, 10000, 50000, 100000, 500000, 1000000])
#n_points = 100000
k_experiments = 100
pa_base = 0.1

comparisons = {
    'means': compare_binomial_sample_means,
    'means_chebyshev': compare_binomial_stderr_chebyshev,
    'means_clt': compare_binomial_clt,
    'max_lk': compare_binomial_maxlikelihood,
    #'bootstrap': compare_groups_bootstrap_binomial,
    'bayes': compare_groups_bayes_binomial,
}

df = pd.DataFrame(columns=['name', 'n_points', 'experiments', 
                           'correct', 'not_sure', 'incorrect'])

for n in n_points:
    exps = gen_binomial_experiments(n, k_experiments, pa_base)
    cmp_exact = compare_experiments_binomial_exact(exps)
    for k, v in comparisons.items():
        cmp = compare_experiments(exps, method_name=k, method_fn=v)
        stat = method_guesses(method=cmp, exact=cmp_exact, n_points=n)
        df = pd.concat([df, pd.DataFrame([stat])], ignore_index=True)

df['correct_and_not_sure'] = df['correct'] + df['not_sure']
df.head(10)

In [ ]:
#todo: make means and max_lk distinguishable
#df['line_dash'] = 0
#df.loc[df['name'] == 'max_lk', 'line_dash'] = 1
#https://stackoverflow.com/questions/64371174/how-to-change-variable-label-names-for-the-legend-in-a-plotly-express-line-chart

fig = px.line(df, x='n_points', y='correct', color='name', markers='markers', log_x=True)
fig.update_layout(
    title='Correct',
    yaxis_title='N Experiments',
    xaxis_title='N Points',
    legend_title='Method',
    yaxis_range=[0, 130],
    height=450, width=800
)
fig.show()

fig = px.line(df, x='n_points', y='correct_and_not_sure', color='name', markers='markers', log_x=True)
fig.update_layout(
    title='Correct and Not Sure',
    yaxis_title='N Experiments',
    xaxis_title='N Points',
    legend_title='Method',
    yaxis_range=[0, 130],
    height=450, width=800
)
fig.show()

*При большом количестве точек решения бутстрапа и байесовского моделирования совпадают.*  
*Бета-распределение и биномиальное распределение переходят в нормальное *
$$
N(p, \frac{p(1-p)}{n}) ?
$$

Бета:  https://en.wikipedia.org/wiki/Beta_distribution#Special_and_limiting_cases  
Биномиальное: https://en.wikipedia.org/wiki/Binomial_distribution#Normal_approximation  

## Ссылки

[[Bern](https://en.wikipedia.org/wiki/Bernoulli_process)] - Bernoulli process, *in Wikipedia*      
[[LLN](https://en.wikipedia.org/wiki/Law_of_large_numbers)] - Law of large numbers, *in Wikipedia*     
[[StErrMn](https://en.wikipedia.org/wiki/Standard_error#Derivation)] - Standard error of the mean, *in Wikipedia*  
[[StdUnbiased](https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation)] - Unbiased estimation of standard deviation, *in Wikipedia*  
[[Cheb](https://en.wikipedia.org/wiki/Chebyshev%27s_inequality)] - Chebyshev's inequality, *in Wikipedia*     
[[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)] - Central limit theorem, *in Wikipedia*   
[[DMLT](https://en.wikipedia.org/wiki/De_Moivre%E2%80%93Laplace_theorem)] - De Moivre-Laplace theorem, *in Wikipedia*   
[[ExpProp](https://en.wikipedia.org/wiki/Expected_value#Properties)] - Expected value, Properties *in Wikipedia*   
[[VarProp](https://en.wikipedia.org/wiki/Standard_deviation#Identities_and_mathematical_properties)] - Standard deviation, Identities and mathematical properties *in Wikipedia*  
[[Cov](https://en.wikipedia.org/wiki/Covariance)] - Covariance, *in Wikipedia*  
[[ConfInt](https://en.wikipedia.org/wiki/Confidence_interval)] - Confidence interval, *in Wikipedia*   
[[NormSum](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables)] - Sum of normally distributed random variables, *in Wikipedia*   
[[MxLk](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation)] - Maximum likelihood estimation, *in Wikipedia*  
[[MAPE](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation)] - Maximum a posteriori estimation, *in Wikipedia*  



# Бутстрап

Еще один вариант закона больших чисел говорит, что по мере набора данных частоты возможных значений в сэмле будут приближаться к их вероятности. Это является основой для частотной интерпретации вероятности.

Также на этом основании имеющийся сэмпл данных можно считать приближением к точному распределению.

Общее название - методы повторной выборки данных или ресэмплинг.  
Один из методов - метод складного ножа (Jackknife).   
Еще один популярный метод - бутстрап.  
Далее обсуждается только последний.

Бутстрап:  
https://projecteuclid.org/journals/annals-of-statistics/volume-7/issue-1/Bootstrap-Methods-Another-Look-at-the-Jackknife/10.1214/aos/1176344552.full

*What Teachers Should Know about the Bootstrap: Resampling in the Undergraduate Statistics Curriculum*  
https://arxiv.org/abs/1411.5279  
https://arxiv.org/pdf/1411.5279.pdf  

Можно оценить неопределенность средних значений.

In [ ]:
def bootstrap_means_rawdata(dt, k_bootstrap):
    bs = np.random.choice(dt, size=(k_bootstrap, n), replace=True)
    means = bs.mean(axis=1)
    return means

def bootstrap_means_binomial(s, n, k_bootstrap):
    p = s/n
    bs = stats.binom.rvs(n, p, size=k_bootstrap)
    means = bs / n
    return means

In [ ]:
p_exact_a = 0.7
p_exact_b = 0.65

n = 5000
s_a = stats.binom.rvs(n, p_exact_a)
s_b = stats.binom.rvs(n, p_exact_b)
p_sample_a = s_a/n
p_sample_b = s_b/n

k_bootstrap = 30000
means_a = bootstrap_means_binomial(s_a, n, k_bootstrap)
means_b = bootstrap_means_binomial(s_b, n, k_bootstrap)

fig = go.Figure()
fig.add_trace(go.Histogram(x=means_a, histnorm='percent',
                           name='A',
                           opacity=0.3))
#fig.add_vline(x=p_sample_a, line_dash='dash')
fig.add_vline(x=p_exact_a)
fig.add_trace(go.Histogram(x=means_b, histnorm='percent',
                           name='B',
                           opacity=0.3))
#fig.add_vline(x=p_sample_b, line_dash='dash')
fig.add_vline(x=p_exact_b)
# fig.add_trace(go.Scatter(x=[p_exact_b, p_exact_b], y=[0, 1], 
#                          mode='lines', line_color='black', line_dash='dash',
#                          name='Exact Mean B'))
fig.update_layout(title='Means Bootstrap Dists',
                  xaxis_title='p',
                  yaxis_title='Prob',
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

In [ ]:
def compare_groups_bootstrap_binomial(sa, na, sb, nb):
    k_bootstrap = 30000
    means_a = bootstrap_means_binomial(sa, na, k_bootstrap)
    means_b = bootstrap_means_binomial(sb, nb, k_bootstrap)
    p_ea_gt_eb = np.sum(means_a > means_b) / len(means_a)
    p_level = 0.95
    res = None
    if p_ea_gt_eb >= p_level:
        res = 'A'
    elif (1 - p_ea_gt_eb) >= p_level:
        res = 'B'
    return res

In [ ]:
p_a_exact = 0.7
p_b_exact = 0.67

n = 1000
sa = stats.binom.rvs(n, p_a_exact)
sb = stats.binom.rvs(n, p_b_exact)

print(f'pa_exact={p_a_exact}, pb_exact={p_b_exact}')
print(f'n={n}, sa={sa}, sb={sb}')
print(f'Exact best group: {"A" if p_a_exact > p_b_exact else "B" if p_b_exact > p_a_exact else None}')
print(f'Group selected by bootstrap comparison: {compare_groups_bootstrap_binomial(sa, n, sb, n)}')

Вычислительно может быть затратно.  
Есть пара трюков для ускорения вычислений.  

Пуассоновский бутстрап:  
https://www.unofficialgoogledatascience.com/2015/08/an-introduction-to-poisson-bootstrap26.html  

Сэмплировать из выборки [x1, x2, ..., xn] с повторением все равно, что генерировать набор
мультиномиальных коэффициетов x ~ Multinomial(n, 1/n). Для больших n предлагается заменить Multinomial(n, 1/n) на n сэмплов из биномиального распределение Binom(n, 1/n). Количество точек в каждом векторе при этом перестает быть одинаковым. Еще один шаг - заменить биномиальное распределение Binom(n, 1/n) на распределение Пуассона Poisson(1). Сэмплировать из него. 

Байесовский бутстрап:  
https://gdmarmerola.github.io/the-bayesian-bootstrap/ - примеры  
https://projecteuclid.org/journalArticle/Download?urlId=10.1214%2Faos%2F1176345338 - оригинальный текст  
https://www.sumsar.net/blog/2015/04/the-non-parametric-bootstrap-as-a-bayesian-model/  

Есть данные x_1, ..., x_n.  
Сгенерировать (n-1) точку u_1, ..., u_{n-1} из равномерного распределениея U(0,1);
Отсортировать их;  
Дополнить точками [0, ... , 1]  
Посчитать разности g_i = u_i - u_i-1, i>=1.  
Использовать g_i как вероятности для [x1, ..., x_n].   
Среднее  
m = sum g_i x_i

Есть отличия в интерпретации от обычного бутстрапа.

Ограничения?  
Бутстрап несколько хуже работает для непрерывных величин.  
При малых размерах исходных данных.  
Если часть данных мало представлена (исходное распределение скошено, в выборке мало значений из хвоста).  